In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder \
.master("local") \
.appName("self_join_df") \
.getOrCreate()

In [10]:
data = [(1,"Smith",1,"10",3000),
    (2,"Rose",1,"20",4000),
    (3,"Williams",1,"10",1000),
    (4,"Jones",2,"10",2000),
    (5,"Brown",2,"40",-1),
    (6,"Brown",2,"50",-1)
       ]

Columns = ("emp_id","name","superior_emp_id","emp_dept_id","salary")
df1 = spark.createDataFrame(data=data, schema=Columns)
df1.show()

+------+--------+---------------+-----------+------+
|emp_id|    name|superior_emp_id|emp_dept_id|salary|
+------+--------+---------------+-----------+------+
|     1|   Smith|              1|         10|  3000|
|     2|    Rose|              1|         20|  4000|
|     3|Williams|              1|         10|  1000|
|     4|   Jones|              2|         10|  2000|
|     5|   Brown|              2|         40|    -1|
|     6|   Brown|              2|         50|    -1|
+------+--------+---------------+-----------+------+



In [15]:
df1.alias("emp1").join(df1.alias("emp2"), \
    col("emp1.superior_emp_id") == col("emp2.emp_id"),"inner") \
    .select(col("emp1.emp_id"),col("emp1.name"), \
      col("emp2.emp_id").alias("superior_emp_id"), \
      col("emp2.name").alias("superior_emp_name")) \
   .show(truncate=False)

+------+--------+---------------+-----------------+
|emp_id|name    |superior_emp_id|superior_emp_name|
+------+--------+---------------+-----------------+
|1     |Smith   |1              |Smith            |
|2     |Rose    |1              |Smith            |
|3     |Williams|1              |Smith            |
|4     |Jones   |2              |Rose             |
|5     |Brown   |2              |Rose             |
|6     |Brown   |2              |Rose             |
+------+--------+---------------+-----------------+



In [16]:
# spark sql self join with where clause
df1.alias("emp1").join(df1.alias("emp2")) \
    .where(col("emp1.superior_emp_id") == col("emp2.emp_id")) \
    .select(col("emp1.emp_id"),col("emp1.name"), \
      col("emp2.emp_id").alias("superior_emp_id"), \
      col("emp2.name").alias("superior_emp_name")) \
   .show(truncate=False)

+------+--------+---------------+-----------------+
|emp_id|name    |superior_emp_id|superior_emp_name|
+------+--------+---------------+-----------------+
|1     |Smith   |1              |Smith            |
|2     |Rose    |1              |Smith            |
|3     |Williams|1              |Smith            |
|4     |Jones   |2              |Rose             |
|5     |Brown   |2              |Rose             |
|6     |Brown   |2              |Rose             |
+------+--------+---------------+-----------------+



In [17]:
# spark sql self join with filter clause
df1.alias("emp1").join(df1.alias("emp2")) \
    .filter(col("emp1.superior_emp_id") == col("emp2.emp_id")) \
    .select(col("emp1.emp_id"),col("emp1.name"), \
      col("emp2.emp_id").alias("superior_emp_id"), \
      col("emp2.name").alias("superior_emp_name")) \
   .show(truncate=False)

+------+--------+---------------+-----------------+
|emp_id|name    |superior_emp_id|superior_emp_name|
+------+--------+---------------+-----------------+
|1     |Smith   |1              |Smith            |
|2     |Rose    |1              |Smith            |
|3     |Williams|1              |Smith            |
|4     |Jones   |2              |Rose             |
|5     |Brown   |2              |Rose             |
|6     |Brown   |2              |Rose             |
+------+--------+---------------+-----------------+



In [18]:
df1.createOrReplaceTempView("emp")
spark.sql("""select e1.emp_id, e1.name, e2.emp_id as Manager_id, e2.name as Manager_name 
from emp e1 inner join emp e2 on e1.superior_emp_id = e2.emp_id""").show()

+------+--------+----------+------------+
|emp_id|    name|Manager_id|Manager_name|
+------+--------+----------+------------+
|     1|   Smith|         1|       Smith|
|     2|    Rose|         1|       Smith|
|     3|Williams|         1|       Smith|
|     4|   Jones|         2|        Rose|
|     5|   Brown|         2|        Rose|
|     6|   Brown|         2|        Rose|
+------+--------+----------+------------+

